In [ ]:
!pip install transformers
!pip install transformers[torch]
!pip install -U sentence-transformers
!pip install annoy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=75b25cb0da21019ff4054b45ae1e6d1b1d70b9722ab074fb3c1f50ace7473629
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 10.6 MB/s eta

In [ ]:
from annoy import AnnoyIndex
import pandas as pd
from sentence_transformers import SentenceTransformer
import time
from tqdm import tqdm
import pickle as pkl

MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'
n_trees = 8096
max_corpus_size = 4000
top_k_hits = 3
annoy_index_path = 'Annoy_index'

def get_data():
    """Распаковка данных и перевод в формат CSV"""
    pd.read_excel('./train_dataset.xlsx').to_csv('QAdata', index=False)
    data = pd.read_csv('./QAdata')
    return list(data.QUESTION)


def new_data():
    """Загрузка данные на сайте (Дополнение датасета)"""
    pd.read_excel('train_dataset.xlsx').to_csv('QAdata', index=False)
    data = pd.read_csv('./QAdata')
    return list(data.QUESTION)

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"{func.__name__} time: {end_time - start_time:.2f} seconds")
        return result
    return wrapper

@timing_decorator
def embedding():
    question = get_data()
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    embeddings = []
    for q in tqdm(question, desc='Embedding'):
        embeddings.append(model.encode(q))
    with open('model.pkl', 'wb') as f:
        pkl.dump(embeddings, f)
    return embeddings

@timing_decorator
def build_annoy_index(embeddings):
    annoy_index = AnnoyIndex(len(embeddings[0]), 'angular')
    for i, emb in tqdm(enumerate(embeddings), desc='Building index', total=len(embeddings)):
        annoy_index.add_item(i, emb)
    annoy_index.build(n_trees)
    annoy_index.save(annoy_index_path)
    return annoy_index


embeddings = embedding()
annoy_index = build_annoy_index(embeddings)

['Что такое Смартократия, и на чем она базируется?', 'Какие принципы лежат в основе Смартократии?', 'Какие основные понятия, правила и процессы управления определяет Кодекс Смартократии?', 'Какие обязанности и ограничения накладывает Кодекс Смартократии на сотрудников ГК?', 'Какие атрибуты включает в себя атрибуты Роли?', 'Какие права и обязанности сотрудника ГК связаны с назначением на Роль?', 'Какие действия могут предпринимать сотрудники ГК, занимающие Роли, в рамках своей Зоны ответственности?', 'Как определяется порядок замещения Роли?', 'Какие Роли избираются, а какие назначаются в ГК?', 'Что представляет собой Круг в контексте Смартократии?', 'Как определяется иерархия кругов?', 'Какие Роли обязательны для каждого Круга?', 'Что представляет собой Круг типа "Сообщество"?', 'Какие Роли избираются в Кругах?', 'Какие процедуры регулируют выборы на избираемые Роли?', 'Какие роли обязательны для каждого Круга?', 'Как определяется Лид-линк Круга, и какие полномочия он имеет?', 'Какие р

Embedding: 100%|██████████| 520/520 [00:15<00:00, 32.51it/s] 


embedding time: 24.84 seconds


Building index: 100%|██████████| 520/520 [00:00<00:00, 20771.20it/s]


build_annoy_index time: 0.87 seconds
